In [ ]:
from pyscf import gto, dft, scf, ao2mo, lib, tddft,fci
from pyscf.fci import addons
import numpy as np
from pyscf.dft import numint
import scipy
from functools import reduce
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import re
from pyscf import tools
from SF_XTDA import *


In [3]:
mol = gto.M(                 
    atom = f'''
    F 0 0 0
    H 0 0 1.0''',
    basis = '6-31G',
    charge = 0,
    spin = 2, 
    verbose = 5,
    #cart=True,
    symmetry='C2v'
)


mf = dft.ROKS(mol)
mf.irrep_nelec = {'A1': (4,2), 'B1':(1,1), 'B2':(1,1)}
mf.xc = 'bhandhlyp'
mf.kernel()

System: uname_result(system='Linux', node='localhost.localdomain', release='4.18.0-348.7.1.el8_5.x86_64', version='#1 SMP Wed Dec 22 13:25:12 UTC 2021', machine='x86_64')  Threads 72
Python 3.9.0 | packaged by conda-forge | (default, Nov 26 2020, 07:57:39) 
[GCC 9.3.0]
numpy 1.26.4  scipy 1.13.1  h5py 3.13.0
Date: Fri Jun 27 09:39:15 2025
PySCF version 2.9.0
PySCF path  /home/lenovo2/anaconda3/envs/py39/lib/python3.9/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 10
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 2
[INPUT] symmetry C2v subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 F      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      0.000000000000   0.00000

double occ irrep_nelec = [2, 1, 1]
single occ irrep_nelec = [2, 0, 0]
                  Roothaan           | alpha              | beta
  Highest 2-occ =   -0.6380416274586 | -0.659228954300253 | -0.618178081948679
  Lowest 0-occ =   0.835730665454669 |  0.753400717934378 |  0.915244296649854
  1-occ =         -0.599561703611245 | -0.864125673800582 | -0.336571221037101
  1-occ =         -0.00291845954444165 | -0.173987884868472 |  0.167764845279417
  Roothaan mo_energy =
[-2.54660496e+01 -1.45924883e+00 -5.99561704e-01 -2.91845954e-03
  8.35730665e-01  1.23581029e+00  1.69416701e+00 -6.38041627e-01
  1.16317420e+00 -6.38041627e-01  1.16317420e+00]
nelec by numeric integration = [6.00000001 4.        ]
    CPU time for vxc      8.70 sec, wall time      0.14 sec
E1 = -147.98261062510682  Ecoul = 54.02221346150753  Exc = -10.742910614072745
cycle= 4 E= -99.940712879392  delta_E= -8.21e-05  |g|= 0.00152  |ddm|= 0.0278
    CPU time for cycle= 4     10.16 sec, wall time      0.17 sec
HOMO (A

-99.94071418791759

In [4]:
nstates = 10
sf_tda = SA_SF_TDA(mf,SA=3)
tmp_e, values = sf_tda.kernel(nstates=nstates)
print(tmp_e) #  in eV
sf_tda.analyse()

nelec by numeric integration = [6.00000001 4.        ]
    CPU time for vxc     10.41 sec, wall time      0.15 sec
nelec by numeric integration = [6.00000001 4.        ]
    CPU time for vxc      9.24 sec, wall time      0.13 sec
omega alpha hyb 0.0 0.5 0.5
GGA
The row of A matrix  42
[-12.24766364  -2.05290235  -2.05290235   0.56596007   2.71600004
  17.20575915  17.20575915  20.59678017  21.92397142  24.57678354]
Excited state 1  -12.24766 eV -100.39080754 Hartree
 98% OO(ab) 6a -> 5b   -0.98886
 
Excited state 2   -2.05290 eV -100.01615696 Hartree
  6% CO(ab) 3a -> 5b   -0.23693
 94% CO(ab) 4a -> 5b    0.96728
 
Excited state 3   -2.05290 eV -100.01615696 Hartree
 94% CO(ab) 3a -> 5b   -0.96728
  6% CO(ab) 4a -> 5b   -0.23693
 
Excited state 4    0.56596 eV -99.91991554 Hartree
 50% OO(ab) 5a -> 5b    0.70614
 50% OO(ab) 6a -> 6b    0.70765
 
Excited state 5    2.71600 eV -99.84090302 Hartree
  3% OV(ab) 6a -> 7b    0.15896
 48% OO(ab) 5a -> 5b    0.69039
 47% OO(ab) 6a -> 6b   -0.6

[-1.9866936081217859,
 -0.9986406163194841,
 -0.9986406163194841,
 -0.0005923889063605969,
 -1.9603862328845088,
 -0.9883319549516875,
 -0.9883319549516876,
 -1.8244681430529632,
 -1.1573551542161862,
 -1.0277665289028737]

In [ ]:
from xtddft.XSF_TDA_GPU import XSF_TDA_GPU

mol = gto.M(
    atom='H 0 0 0; F 0 0 1.0',
    basis = 'cc-pvdz',
    charge=0,
    spin=2,
    verbose=4,
    # symmetry = 'D2h',
)
mf = dft.ROKS(mol)
mf.xc = 'b3lyp'
mf.kernel()
sf_tda = XSF_TDA_GPU(mf, davidson=True, collinear='mcol')  # use davidson iteration and multicollinear kernel
e0, values = sf_tda.kernel(nstates=10, gpu_davidson=False)  # calculate 10 excited states and use pyscf davidson instead of gpu4pyscf
print('excited energy ', e0)